## Loading the dataset
### Imports

In [96]:
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Input

### Convert filename to useful metadata

In [79]:
def filename_to_metadata(file_names: list[str], file_extension: str, only_labels=False):
    """
    Gets dataset type, fold, source name, label and take from .wav filename.

    :param file_names: List of .wav filename strings
    :param file_extension: The file extension (e.g. .wav).
    :param only_labels: Bool, where True means returning only a list of labels.
    :return: List of dictionaries ["dataset_type", "fold", "source_name", "label", "take"] or strings.
    """
    # stft-1-137-A-32.wav
    # stft --> Dataset type
    # 1 --> Fold
    # 137 --> Source name
    # A --> Take
    # 32 --> Label

    metadata_list = []

    for filename in file_names:
        filename = filename.replace(file_extension, "")
        dataset_type, fold, source_name, take, label = re.split(r"-", filename)

        data_dict = {"dataset_type": dataset_type, "fold": int(fold), "source_name": source_name, "take": take,
                     "label": int(label)}

        if only_labels:
            metadata_list.append(data_dict["label"])
        else:
            metadata_list.append(data_dict)

    return metadata_list

In [80]:
def filepath_to_img_data(path_to_dir: Path, filenames: list[str], resize_to_square=False):
    """
    Takes the path to images to create data objects.

    :param path_to_dir: Path object of the path to the data directory.
    :param filenames: List of filenames.
    :param resize_to_square: True means resizing the image to have the same width and height.
    :return:
    """
    img_data = []

    for filename in filenames:
        path_to_img = path_to_dir / filename

        base_img = cv2.imread(str(path_to_img))
        final_img = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGB)

        if resize_to_square:
            new_img_size = max(final_img.shape)

            final_img = cv2.resize(final_img, (new_img_size, new_img_size))

        img_data.append(final_img)

    return img_data

### Relative paths

In [81]:
path_to_data = Path.cwd() / ".." / "data"

path_to_waveform = path_to_data / "waveform"
path_to_stft = path_to_data / "STFT"
path_to_mel = path_to_data / "mel"

### Filenames of all images

In [82]:
waveform_filenames = [file.name for file in path_to_waveform.glob("*.png")]
stft_filenames = [file.name for file in path_to_stft.glob("*.png")]
mel_filenames = [file.name for file in path_to_mel.glob("*.png")]

### Load images as dataset X and y

In [85]:
wav_X = filepath_to_img_data(path_to_waveform, waveform_filenames)
wav_y = filename_to_metadata(waveform_filenames, ".png", only_labels=True)

stft_X = filepath_to_img_data(path_to_stft, stft_filenames)
stft_y = filename_to_metadata(stft_filenames, ".png", only_labels=True)

mel_X = filepath_to_img_data(path_to_mel, mel_filenames)
mel_y = filename_to_metadata(mel_filenames, ".png", only_labels=True)

### Train/test split

In [90]:
def split_and_nparray(X_set, y_set, _train_size, _rng):
    X_train, X_test, y_train, y_test = train_test_split(
        X_set, y_set, train_size=_train_size, random_state=_rng
    )

    return map(np.array, (X_train, X_test, y_train, y_test))


train_size = 0.9
rng = 42

wav_X_train, wav_X_test, wav_y_train, wav_y_test = split_and_nparray(wav_X, wav_y, train_size, rng)
stft_X_train, stft_X_test, stft_y_train, stft_y_test = split_and_nparray(stft_X, stft_y, train_size, rng)
mel_X_train, mel_X_test, mel_y_train, mel_y_test = split_and_nparray(mel_X, mel_y, train_size, rng)

In [95]:
num_train_samples = wav_X_train.shape[0]
img_size = wav_X_train.shape[1:3]
num_channels = wav_X_train.shape[3]
num_classes = 50

print(f"Image size (width x height): {img_size[1]} x {img_size[0]}")

Image size (width x height): 496 x 369


## Defining the CNN model

### Parameters:
* Classes = 50
* Channels = 3
* Kernel size
* Padding
* Stride
* Dilation

### Layers:
* Pooling (max pooling and average pooling)
* Strided conv
* Normalization
* Activation (ReLU)
* Conv --> Normalization --> Activation

### Dense:
* Regular NN